In [1]:
!pip install mlflow


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import mlflow


from scipy.stats import (
                         shapiro,
                         levene,
                         mannwhitneyu,
                         ttest_ind,
                         chi2_contingency,
                         fisher_exact)

from sklearn.preprocessing import (
                            MinMaxScaler,
                            OneHotEncoder,
                            LabelEncoder
                            )

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('display.max_columns', None)

In [3]:
root = Path('Attrition_Application/artifacts/data_ingestion_artifact/Attrition.csv')
dataset_path = Path.joinpath(Path('/').resolve(),root)

In [4]:
df = pd.read_csv(dataset_path)

In [5]:
df.sample(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
278,26,No,Travel_Frequently,1479,Research & Development,1,3,Life Sciences,1,384,3,Female,84,3,2,Manufacturing Director,2,Divorced,6397,26767,1,Y,No,20,4,1,80,1,6,6,1,6,5,1,4
1002,35,No,Travel_Frequently,200,Research & Development,18,2,Life Sciences,1,1412,3,Male,60,3,3,Manufacturing Director,4,Single,9362,19944,2,Y,No,11,3,3,80,0,10,2,3,2,2,2,2
511,36,No,Travel_Rarely,913,Research & Development,9,2,Medical,1,699,2,Male,48,2,2,Manufacturing Director,2,Divorced,8847,13934,2,Y,Yes,11,3,3,80,1,13,2,3,3,2,0,2
787,55,No,Travel_Frequently,1091,Research & Development,2,1,Life Sciences,1,1096,4,Male,65,3,3,Manufacturing Director,2,Married,10976,15813,3,Y,No,18,3,2,80,1,23,4,3,3,2,1,2
1304,40,No,Travel_Rarely,750,Research & Development,12,3,Life Sciences,1,1829,2,Female,47,3,2,Healthcare Representative,1,Divorced,4448,10748,2,Y,No,12,3,2,80,1,15,3,3,7,4,7,7


In [4]:
df.shape

(1470, 35)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [6]:
df.isna().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [7]:
df.select_dtypes(exclude='object').describe().T

,count,mean,std,min,25%,50%,75%,max
Age,1470.0,36.923810,9.135373,18.0,30.00,36.0,43.00,60.0
DailyRate,1470.0,802.485714,403.509100,102.0,465.00,802.0,1157.00,1499.0
DistanceFromHome,1470.0,9.192517,8.106864,1.0,2.00,7.0,14.00,29.0
Education,1470.0,2.912925,1.024165,1.0,2.00,3.0,4.00,5.0
EmployeeCount,1470.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0
EmployeeNumber,1470.0,1024.865306,602.024335,1.0,491.25,1020.5,1555.75,2068.0
EnvironmentSatisfaction,1470.0,2.721769,1.093082,1.0,2.00,3.0,4.00,4.0
HourlyRate,1470.0,65.891156,20.329428,30.0,48.00,66.0,83.75,100.0
JobInvolvement,1470.0,2.729932,0.711561,1.0,2.00,3.0,3.00,4.0
JobLevel,1470.0,2.063946,1.106940,1.0,1.00,2.0,3.00,5.0


In [8]:
num_features = df.select_dtypes(exclude='object').columns.to_list()
cat_features = df.select_dtypes(include='object').columns.to_list()

descrete_features = list()
continues_features = list()

for feature in num_features:
    if df[feature].nunique() > 5:
        continues_features.append(feature)
    else:
        descrete_features.append(feature)    

cat_features.remove('Attrition')

target_feature = 'Attrition'

In [9]:
significant_num_features = list()

def checkNormality(sample_1,sample_2):
    print('Shapiro-Wilk Test')
    # Null Hypothesis (H0): the data was drawn from a normal distribution.
    # Alternative Hypothesis (H1): the data was not drawn from a normal distribution.

    stats_1,p_val_1 = shapiro(sample_1)
    stats_2,p_val_2 = shapiro(sample_2)
    print()
    print(f'sample 1 : stats {stats_1}, p-val: {p_val_1}')
    print(f'sample 1 : stats {stats_2}, p-val: {p_val_2}')
    result = p_val_2 > 0.05 or p_val_1 > 0.05
    return result


def checkVarianceHomogenisity(sample_1,sample_2):
    print('levene Test')
    # Null Hypothesis (H0): The variances are equal across the groups.
    # Alternative Hypothesis (H1): The variances are not equal across the groups.

    stats, p_val = levene(sample_1, sample_2)
    print()
    print(f'sample 1 : stats {stats}, p-val: {p_val}')
    result = p_val > 0.05
    return result


def MannWhitneyTest(sample_1, sample_2):
    print('Mann-Whitney U test')
    # Null Hypothesis (H0): The distributions of the two groups are equal.
    # Alternative Hypothesis (H1): The distributions of the two groups are not equal.

    stats, p_val = mannwhitneyu(sample_1, sample_2)
    print()
    print(f'sample 1 : stats {stats}, p-val: {p_val}')
    result = p_val > 0.05
    return result
        

def Ttest(sample_1, sample_2,var_same=True):
    if var_same:
        print('T-test of Independence')
    else:
        print("Wetch's T-test")    
    # Null Hypothesis (H0): The distributions of the two groups are equal.
    # Alternative Hypothesis (H1): The distributions of the two groups are not equal.

    stats, p_val = ttest_ind(sample_1, sample_2,equal_var=var_same)
    print()
    print(f'sample 1 : stats {stats}, p-val: {p_val}')
    result = p_val > 0.05
    return result


def CheckSignificance(feature):
    print('levene Test')
    # taking out random samples
    class_samples_1 = df[df['Attrition'] == 'Yes'][feature].sample(200)
    class_samples_2 = df[df['Attrition'] == 'No'][feature].sample(200)
    print(feature)
    print('-'*25)

    

    if checkNormality(class_samples_1,class_samples_2):
        print('The samples not follows normal distribution. (Reject H0) ✓')
        if checkVarianceHomogenisity(class_samples_1,class_samples_2):
            print(f'This variance of two given samples are homogeneus. (Reject H0) ✓') 
            if Ttest(class_samples_1, class_samples_2):
                print(f'This means that there is a significant difference between the two groups. (Reject H0) ✓')  
                significant_num_features.append(feature)     
            else:
                 print(f'This means that there is no significant difference between the two groups. (Accept H0) ✗')   
        else:
            if MannWhitneyTest(class_samples_1, class_samples_2):
                print(f'This means that there is a significant difference between the two groups. (Reject H0) ✓')  
                significant_num_features.append(feature)     
            else:
                print(f'This means that there is no significant difference between the two groups. (Accept H0) ✗')
    else:
        print('The samples follows normal distribution. (Reject H0) ✓')
        if checkVarianceHomogenisity(class_samples_1,class_samples_2):
            print(f'This variance of two given samples are homogeneus. (Reject H0) ✓') 
            if Ttest(class_samples_1, class_samples_2,False):
                print(f'This means that there is a significant difference between the two groups. (Reject H0) ✓')  
                significant_num_features.append(feature)     
            else:
                 print(f'This means that there is no significant difference between the two groups. (Accept H0) ✗')   
        else:
            if MannWhitneyTest(class_samples_1, class_samples_2):
                print(f'This means that there is a significant difference between the two groups. (Reject H0) ✓')  
                significant_num_features.append(feature)     
            else:
                print(f'This means that there is no significant difference between the two groups. (Accept H0) ✗')
    print('-'*25)

for feature in continues_features:
    CheckSignificance(feature)

levene Test
Age
-------------------------
Shapiro-Wilk Test

sample 1 : stats 0.9436856882738741, p-val: 4.863090252907193e-07
sample 1 : stats 0.9705367516738206, p-val: 0.00032889245311862206
The samples follows normal distribution. (Reject H0) ✓
levene Test

sample 1 : stats 0.10198082950836605, p-val: 0.749632934099071
This variance of two given samples are homogeneus. (Reject H0) ✓
Wetch's T-test

sample 1 : stats -3.8022850757513713, p-val: 0.00016596890715205086
This means that there is no significant difference between the two groups. (Accept H0) ✗
-------------------------
levene Test
DailyRate
-------------------------
Shapiro-Wilk Test

sample 1 : stats 0.9517775540542107, p-val: 2.810291374744542e-06
sample 1 : stats 0.949848987316726, p-val: 1.824735320650447e-06
The samples follows normal distribution. (Reject H0) ✓
levene Test

sample 1 : stats 0.30900139607183175, p-val: 0.5786059650186048
This variance of two given samples are homogeneus. (Reject H0) ✓
Wetch's T-test



In [10]:
for feature in descrete_features:
    print(feature,df[feature].nunique())

descrete_features = [feature for feature in descrete_features if df[feature].nunique() > 1]  

Education 5
EmployeeCount 1
EnvironmentSatisfaction 4
JobInvolvement 4
JobLevel 5
JobSatisfaction 4
PerformanceRating 2
RelationshipSatisfaction 4
StandardHours 1
StockOptionLevel 4
WorkLifeBalance 4


In [11]:
for feature in cat_features:
    print(feature,df[feature].nunique())
    
cat_features = [feature for feature in cat_features if df[feature].nunique() > 1]  

BusinessTravel 3
Department 3
EducationField 6
Gender 2
JobRole 9
MaritalStatus 3
Over18 1
OverTime 2


In [12]:
significant_cat_features = list()

def significant_bin_cat_features(feature,target):
    print(feature)
    feat,target = df[feature].sample(500),df[target].sample(500)
    contingency_table = pd.crosstab(feat,target)
    print('-'*25)

    result = chi2_contingency(contingency_table)
    if (result.expected_freq < 5).any():
        p_value = result.pvalue
        print(f'The Frequency count greater than 5. Assumption satisfied ✓') 
        if p_value < 0.05:
            print("Reject the null hypothesis: There is an association between the two categorical variables. ✓")
            significant_cat_features.append(feature)
        else:
            print("Fail to reject the null hypothesis: There is no association between the two categorical variables. ✗") 
    else:
        print(f'The Frequency count is greater than 5. Assumption violated ✗')
        print()
        print('Fisher exact test')

        # Null Hypothesis (H0): There is no association between the two categorical variables.
        # Alternative Hypothesis (H1): There is an association between the two categorical variables.
        oddsratio, p_value = fisher_exact(contingency_table)

        print("Odds Ratio:", oddsratio)
        print("P-value:", p_value)

        # Interpretation based on p-value
        if p_value < 0.05:
            print("Reject the null hypothesis: There is an association between the two categorical variables. ✓")
            significant_cat_features.append(feature)
        else:
            print("Fail to reject the null hypothesis: There is no association between the two categorical variables. ✗")     
    print('-'*25) 


for feature in descrete_features+cat_features:
    significant_bin_cat_features(feature,target_feature)       
        

Education
-------------------------
The Frequency count greater than 5. Assumption satisfied ✓
Fail to reject the null hypothesis: There is no association between the two categorical variables. ✗
-------------------------
EnvironmentSatisfaction
-------------------------
The Frequency count greater than 5. Assumption satisfied ✓
Fail to reject the null hypothesis: There is no association between the two categorical variables. ✗
-------------------------
JobInvolvement
-------------------------
The Frequency count greater than 5. Assumption satisfied ✓
Reject the null hypothesis: There is an association between the two categorical variables. ✓
-------------------------
JobLevel
-------------------------
The Frequency count greater than 5. Assumption satisfied ✓
Reject the null hypothesis: There is an association between the two categorical variables. ✓
-------------------------
JobSatisfaction
-------------------------
The Frequency count is greater than 5. Assumption violated ✗

Fisher

In [13]:
final_selected_features = significant_cat_features+significant_num_features+ [target_feature]

In [14]:
final_selected_features

['JobInvolvement',
 'JobLevel',
 'StockOptionLevel',
 'JobRole',
 'MaritalStatus',
 'OverTime',
 'DistanceFromHome',
 'EmployeeNumber',
 'HourlyRate',
 'MonthlyRate',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'TrainingTimesLastYear',
 'YearsSinceLastPromotion',
 'Attrition']

In [15]:
data = df[final_selected_features]

In [16]:
sacle_features= [
                 'HourlyRate',
                 'MonthlyRate',
                 'PercentSalaryHike',
                 'TrainingTimesLastYear',
                 'YearsSinceLastPromotion',
                 'EmployeeNumber',
                 'JobInvolvement',
                 'JobSatisfaction',
                 'RelationshipSatisfaction']
ohe_features = ['OverTime']
label_encode_features = ['Attrition']

In [17]:
mx = MinMaxScaler()
ohe = OneHotEncoder(dtype=int,drop='first')
le = LabelEncoder()


ct = ColumnTransformer([
   ('one_hot_encoding',ohe,ohe_features),
   ('min_max_scaling',mx,sacle_features)]
 )

X , y = df.drop('Attrition',axis=1),df['Attrition']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_transformed = ct.fit_transform(X_train)
X_test_transformed = ct.transform(X_test)

y_train_transformed = le.fit_transform(y_train)
y_test_transformed = le.transform(y_test)

columns =[feature.split('__')[-1] for feature in ct.get_feature_names_out()]
X_train_transformed = pd.DataFrame(X_train_transformed,columns=columns)
X_test_transformed = pd.DataFrame(X_test_transformed,columns=columns)

y_train_transformed = pd.DataFrame(y_train_transformed,dtype=float)
y_test_transformed = pd.DataFrame(y_test_transformed,dtype=float)

In [168]:
lr1 = Logit(y_train_transformed,X_train_transformed,).fit(maxiter=100)
lr1.summary()

Optimization terminated successfully.
         Current function value: 0.402030
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      0   No. Observations:                  984
Model:                          Logit   Df Residuals:                      973
Method:                           MLE   Df Model:                           10
Date:                Fri, 28 Feb 2025   Pseudo R-squ.:                  0.1173
Time:                        16:40:52   Log-Likelihood:                -395.60
converged:                       True   LL-Null:                       -448.15
Covariance Type:            nonrobust   LLR p-value:                 5.170e-18
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -0.3534      0.484     -0.730      0.465      -1.302       0.595
OverTime_Yes                 1.5082      0.183      8.222      0.000       1.149       1.868
HourlyRate                  -0.0421      0.319     -0.132      0.895      -0.668       0.584
MonthlyRate                  0.0295      0.321      0.092      0.927      -0.599       0.658
PercentSalaryHike           -0.0777      0.353     -0.220      0.826      -0.769       0.614
TrainingTimesLastYear       -0.5293      0.444     -1.192      0.233      -1.400       0.341
YearsSinceLastPromotion     -0.8450      0.484     -1.747      0.081      -1.793       0.103
EmployeeNumber               0.2318      0.309      0.750      0.453      -0.374       0.837
JobInvolvement              -1.6874      0.372     -4.535      0.000      -2.417      -0.958
JobSatisfaction             -0.7788      0.244     -3.188      0.001      -1.258      -0.300
RelationshipSatisfaction    -0.3959      0.254     -1.558      0.119      -0.894       0.102
============================================================================================
"""

In [199]:
exp = mlflow.create_experiment(name='LogitModelExperiment1')

In [207]:
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
exp = mlflow.create_experiment(name=exp)

with mlflow.start_run():
    lr1 = Logit(y_train_transformed,X_train_transformed,).fit(maxiter=100)
    mlflow.statsmodels.log_model(lr1,artifact_path='Statsmodels')
    params = lr1.pvalues.to_dict()
    mlflow.log_params(params)

Optimization terminated successfully.
         Current function value: 0.402030
         Iterations 6


2025/02/28 19:03:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run agreeable-shad-437 at: http://127.0.0.1:5000/#/experiments/0/runs/a4e070f82018455181e7beeea18d10b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


{'const': 0.4654069279460823,
 'OverTime_Yes': 1.9994468407938194e-16,
 'HourlyRate': 0.8951964558753169,
 'MonthlyRate': 0.9268233717923257,
 'PercentSalaryHike': 0.8256381607834216,
 'TrainingTimesLastYear': 0.23344689394942575,
 'YearsSinceLastPromotion': 0.0806646828027661,
 'EmployeeNumber': 0.453227827523358,
 'JobInvolvement': 5.758670029444156e-06,
 'JobSatisfaction': 0.0014325374944757702,
 'RelationshipSatisfaction': 0.11915627868283553}

In [173]:
VIF = pd.DataFrame()
VIF['Features'] = X_train_transformed.columns.to_list()
VIF['VIF'] = [variance_inflation_factor(X_train_transformed.values,i) for i in range(X_train_transformed.shape[1])]
VIF.sort_values(by='VIF',ascending=False,inplace=True)
VIF 

,Features,VIF
0,const,29.894225
2,HourlyRate,1.017786
6,YearsSinceLastPromotion,1.012729
9,JobSatisfaction,1.010868
7,EmployeeNumber,1.010062
10,RelationshipSatisfaction,1.009019
1,OverTime_Yes,1.008058
8,JobInvolvement,1.005936
5,TrainingTimesLastYear,1.005830
4,PercentSalaryHike,1.004683
